# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [3]:
# as dict
dict(vocs)

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [4]:
#  re-parse dict
vocs2 = VOCS.from_dict(dict(vocs))

In [5]:
# Check that these are the same
vocs2 == vocs

True

In [6]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [7]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [8]:
# json
vocs.to_json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [9]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,188.495960,-0.065599,1234,188.430361,188.561560,376.860721,377.123119
6,956.238838,-0.360246,1234,955.878592,956.599084,1911.757184,1913.198168
7,685.626182,0.106777,1234,685.732959,685.519405,1371.465918,1371.038810
8,666.166603,-0.334063,1234,665.832539,666.500666,1331.665078,1333.001332
9,445.981584,-0.375737,1234,445.605847,446.357321,891.211694,892.714642
10,252.716591,-0.352993,1234,252.363598,253.069584,504.727196,506.139167
11,970.599732,0.084972,1234,970.684704,970.514759,1941.369408,1941.029519
12,406.904574,-0.456882,1234,406.447692,407.361456,812.895385,814.722913
13,558.119903,0.985838,1234,559.105741,557.134065,1118.211482,1114.268131
14,323.143112,-0.563229,1234,322.579883,323.706342,645.159766,647.412683


In [10]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [11]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-188.430361,188.561560
6,-955.878592,956.599084
7,-685.732959,685.519405
8,-665.832539,666.500666
9,-445.605847,446.357321
10,-252.363598,253.069584
11,-970.684704,970.514759
12,-406.447692,407.361456
13,-559.105741,557.134065
14,-322.579883,323.706342


In [12]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-188.430361,188.561560
6,-955.878592,956.599084
7,-685.732959,685.519405
8,-665.832539,666.500666
9,-445.605847,446.357321
10,-252.363598,253.069584
11,-970.684704,970.514759
12,-406.447692,407.361456
13,-559.105741,557.134065
14,-322.579883,323.706342


In [13]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-188.43036068,  188.56155962],
       [-955.87859204,  956.59908406],
       [-685.73295892,  685.51940521],
       [-665.83253909,  666.50066602],
       [-445.60584696,  446.35732104],
       [-252.36359809,  253.0695836 ],
       [-970.68470408,  970.51475927],
       [-406.44769233,  407.36145629],
       [-559.1057412 ,  557.13406545],
       [-322.57988313,  323.70634153]])

In [14]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,374.860721,-377.123119
6,1909.757184,-1913.198168
7,1369.465918,-1371.038810
8,1329.665078,-1333.001332
9,889.211694,-892.714642
10,502.727196,-506.139167
11,1939.369408,-1941.029519
12,810.895385,-814.722913
13,1116.211482,-1114.268131
14,643.159766,-647.412683


In [15]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


In [16]:
# normalize inputs to unit domain [0,1]
normed_data = vocs.normalize_inputs(data)
normed_data

,a,b
5,0.188496,0.467200
6,0.956239,0.319877
7,0.685626,0.553388
8,0.666167,0.332968
9,0.445982,0.312131
10,0.252717,0.323504
11,0.970600,0.542486
12,0.406905,0.271559
13,0.558120,0.992919
14,0.323143,0.218385


In [17]:
# and denormalize
vocs.denormalize_inputs(normed_data)

,a,b
5,188.495960,-0.065599
6,956.238838,-0.360246
7,685.626182,0.106777
8,666.166603,-0.334063
9,445.981584,-0.375737
10,252.716591,-0.352993
11,970.599732,0.084972
12,406.904574,-0.456882
13,558.119903,0.985838
14,323.143112,-0.563229


# Error handling

In [18]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [19]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [20]:
data['a']  = np.nan

In [21]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [22]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [23]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
